In [1]:
!pip install datasets
!pip install transformers

Looking in indexes: http://192.168.16.2:8080/root/pypi/+simple/, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
Looking in indexes: http://192.168.16.2:8080/root/pypi/+simple/, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [2]:
!pip install accelerate

Looking in indexes: http://192.168.16.2:8080/root/pypi/+simple/, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [3]:
!pip install evaluate
!pip install rouge_score

Looking in indexes: http://192.168.16.2:8080/root/pypi/+simple/, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
Looking in indexes: http://192.168.16.2:8080/root/pypi/+simple/, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [4]:
!pip install nltk

Looking in indexes: http://192.168.16.2:8080/root/pypi/+simple/, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [5]:
!pip install wandb

Looking in indexes: http://192.168.16.2:8080/root/pypi/+simple/, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [6]:
#wandb login
#db70f6e07a19af66c38b06a7e0400f66ca83495d

In [7]:
import pandas as pd
import numpy as np
import torch

In [8]:
query_response_df = pd.read_csv("Datasets_user/query_response_tech_multi(20).csv", encoding='utf-8', low_memory=False)

In [9]:
print(query_response_df.shape)
query_response_df.head()

(179338, 14)


,tweet_id_x,author_id_x,inbound_x,created_at_x,text_x,response_tweet_id_x,in_response_to_tweet_id_x,tweet_id_y,author_id_y,inbound_y,created_at_y,text_y,response_tweet_id_y,in_response_to_tweet_id_y
0,8.0,115712,True,Tue Oct 31 21:45:10 +0000 2017,is the worst customer service,"9,6,10",NaN,9.0,sprintcare,False,Tue Oct 31 21:46:14 +0000 2017,I would love the chance to review the account ...,NaN,8.0
1,8.0,115712,True,Tue Oct 31 21:45:10 +0000 2017,is the worst customer service,"9,6,10",NaN,10.0,sprintcare,False,Tue Oct 31 21:45:59 +0000 2017,Hello! We never like our customers to feel lik...,NaN,8.0
2,18.0,115713,True,Tue Oct 31 19:56:01 +0000 2017,y’all lie about your “great” connection. 5 bar...,17,NaN,17.0,sprintcare,False,Tue Oct 31 19:59:13 +0000 2017,H there! We'd definitely like to work with you...,16,18.0
3,29.0,115716,True,Tue Oct 31 22:01:35 +0000 2017,actually that's a broken link you sent me and ...,28,NaN,28.0,Ask_Spectrum,False,Tue Oct 31 22:05:37 +0000 2017,The information pertaining to the account assu...,24,29.0
4,31.0,115717,True,Tue Oct 31 22:06:54 +0000 2017,Yo your customer service reps are super nice— ...,30,NaN,30.0,Ask_Spectrum,False,Tue Oct 31 22:12:09 +0000 2017,"Hello, My apologies for any frustrations or in...",NaN,31.0


In [10]:
query_response_df[query_response_df["text_x"].isna()]

,tweet_id_x,author_id_x,inbound_x,created_at_x,text_x,response_tweet_id_x,in_response_to_tweet_id_x,tweet_id_y,author_id_y,inbound_y,created_at_y,text_y,response_tweet_id_y,in_response_to_tweet_id_y


In [11]:
query_response_df[query_response_df["text_y"].isna()]

,tweet_id_x,author_id_x,inbound_x,created_at_x,text_x,response_tweet_id_x,in_response_to_tweet_id_x,tweet_id_y,author_id_y,inbound_y,created_at_y,text_y,response_tweet_id_y,in_response_to_tweet_id_y


In [12]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

modelname = "microsoft/GODEL-v1_1-large-seq2seq"

#Initialize tokenizer and model
print("loading model...", end='', flush=True)
tokenizer = AutoTokenizer.from_pretrained(modelname)
model = AutoModelForSeq2SeqLM.from_pretrained(modelname)
print("completed")

loading model...completed


In [13]:
from datasets import Dataset
#subset_df = (query_response_df.sample(n = 10000, random_state = 23))
#train-eval-test split
train_df = (query_response_df.sample(frac = 0.99, random_state = 23))[["text_x","text_y"]]
temp_df = (query_response_df.drop(train_df.index))[["text_x","text_y"]]
eval_df = (temp_df.sample(frac = 0.5, random_state = 23))[["text_x","text_y"]]
test_df = (temp_df.drop(eval_df.index))[["text_x","text_y"]]
trainset = Dataset.from_pandas(train_df)
evalset = Dataset.from_pandas(eval_df)
testset = Dataset.from_pandas(test_df)

In [14]:
train_df.to_csv("Datasets_user/query_response_tech_multi(20)_train.csv", index=False, encoding="utf-8")
eval_df.to_csv("Datasets_user/query_response_tech_multi(20)_eval.csv", index=False, encoding="utf-8")
test_df.to_csv("Datasets_user/query_response_tech_multi(20)_test.csv", index=False, encoding="utf-8")

In [15]:
trainset = trainset.remove_columns(["__index_level_0__"])
evalset = evalset.remove_columns(["__index_level_0__"])
testset = testset.remove_columns(["__index_level_0__"])

In [16]:
trainset

Dataset({
    features: ['text_x', 'text_y'],
    num_rows: 177545
})

In [17]:
trainset[10]

{'text_x': "I can't sign into https://t.co/mnaWejo8dK from Google Chrome Version 62.0.3202.94 (Official Build) (64-bit) on a Windows 10 PC. HALP!",
 'text_y': "We want to be sure you can access https://t.co/tmRhWEkIq4 with no issues. Let's get some details. What happens when you try to sign in?"}

In [18]:
#function to generate input to be fed into model
def dataset_mapping_function(examples):
    contextes = examples['text_x']
    responses = examples['text_y']
    kbs = 'none'
    
    inputs = []
    for context, kb in zip(contextes, kbs):
        if kb == 'none':
            inputs.append(context + ' => ')
        else:
            _input = context + ' <|Knowledge|> ' + kb + ' => '
            inputs.append(_input)
    model_inputs = tokenizer(inputs, max_length=64, padding="max_length", truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(responses, max_length=64, padding="max_length", truncation=True)
        
    model_inputs["labels"] = labels["input_ids"]
    
    return model_inputs

In [19]:
trainset.column_names

['text_x', 'text_y']

In [20]:
column_names = trainset.column_names

tokenized_trainset = trainset.map(
    dataset_mapping_function,
    batched=True,
    batch_size = 4,
    remove_columns=column_names,
    desc=f"Processing dataset",
)

tokenized_evalset = evalset.map(
    dataset_mapping_function,
    batched=True,
    batch_size = 4,
    remove_columns=column_names,
    desc=f"Processing dataset"
)

tokenized_testset = testset.map(
    dataset_mapping_function,
    batched=True,
    batch_size = 4,
    remove_columns=column_names,
    desc=f"Processing dataset"
)

Processing dataset:   0%|          | 0/177545 [00:00<?, ? examples/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3606: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Processing dataset:   0%|          | 0/896 [00:00<?, ? examples/s]

Processing dataset:   0%|          | 0/897 [00:00<?, ? examples/s]

In [21]:
tokenized_trainset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 177545
})

In [22]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    tokenizer = tokenizer,
    model=model,
)

In [23]:
import evaluate

rouge = evaluate.load("rouge")

def compute_metrics(output):
    pred = np.argmax(output.predictions[0], axis=-1)
    pred = tokenizer.batch_decode(pred, skip_special_tokens=True)
    labels = output.label_ids
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    #rougeL: recall-oriented metric based on the longest common subsequence
    rouge_output = rouge.compute(predictions=pred, references=labels, rouge_types=["rougeL"])

    return rouge_output

In [24]:
import torch
import gc
gc.collect()
torch.cuda.empty_cache()

In [25]:
from numba import cuda
device = cuda.get_current_device()
device.reset()

In [26]:
import os 
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [27]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Thu Jun  8 03:20:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 12.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A40          Off  | 00000000:25:00.0 Off |                    0 |
|  0

In [29]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='trainer', 
    learning_rate=5e-5, 
    num_train_epochs=3,
    evaluation_strategy = 'steps', #unable to run validation without kernel crashing
    save_strategy='steps',
    auto_find_batch_size=False, 
    per_device_train_batch_size=64,
    per_device_eval_batch_size=2,
    eval_accumulation_steps=100,
    optim='adafactor', 
    logging_steps=500,
    save_steps=500,
    save_total_limit=3,
    load_best_model_at_end=True,
    report_to='wandb',
    #resume_from_checkpoint='trainer/checkpoint-13000'
)

trainer = Trainer(
    model,
    training_args,
    train_dataset = tokenized_trainset,
    eval_dataset = tokenized_evalset,
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics
)

trainer.train()
#training loss = cross-entropy loss

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: waihoe-wwh. Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Rougel
500,1.169900,0.950680,0.528229
1000,0.982000,0.876142,0.552812
1500,0.932200,0.840944,0.565199
2000,0.903300,0.814757,0.573293
2500,0.879000,0.799540,0.579400
3000,0.852800,0.783528,0.587967
3500,0.834500,0.773407,0.591382
4000,0.830800,0.764768,0.594986
4500,0.813900,0.757744,0.596000
5000,0.810800,0.751415,0.599133


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



TrainOutput(global_step=8325, training_loss=0.8548559368694867, metrics={'train_runtime': 11745.43, 'train_samples_per_second': 45.348, 'train_steps_per_second': 0.709, 'total_flos': 1.4414749581312e+17, 'train_loss': 0.8548559368694867, 'epoch': 3.0})

In [30]:
trainer.save_model("GODEL_twcs_multi(20)")

In [31]:
predictions = trainer.predict(tokenized_testset)

In [32]:
preds_token = np.argmax(predictions.predictions[0], axis=-1)

In [33]:
print(preds_token[2])

[   27 22103  1114    12  1130    95    28    25    11   217     3    99
    25   130   341   578   807    28    39   364     5     1 32101 32101
 32101 32101 32101 32101 32101 32101 32101 32101 32101 32101 32101 32101
 32101 32101 32101 32101 32101 32101 32101 32101 32101 32101 32101 32101
 32101 32101 32101 32101 32101 32101 32101 32101 32101 32101 32101 32101
 32101 32101 32101 32101]


In [34]:
preds = tokenizer.batch_decode(preds_token, skip_special_tokens=True)

In [35]:
labels = np.where(predictions.label_ids != -100, predictions.label_ids, tokenizer.pad_token_id)

In [36]:
refs = tokenizer.batch_decode(labels, skip_special_tokens=True)

In [37]:
print(testset[2]['text_x'])
print(testset[2]['text_y'])
print(preds[2])

Bruh. needs to get their stuff together. There’s nothing worse than the WiFi going out during a bingeathon 
I just wanted to follow up with you and see if you were still having issues with your services. 
I apologize wanted to follow up with you and see if you were still having issues with your services.


In [38]:
import evaluate

metric = evaluate.load("rouge")
metric.compute(predictions=preds, references=refs)

{'rouge1': 0.6127313098799714,
 'rouge2': 0.38813248368618103,
 'rougeL': 0.5939016813399336,
 'rougeLsum': 0.594131985186217}